# Aetheria — Feedback Retrain Notebook

**Purpose:** Take `feedback.jsonl` you collected locally with Envy teach mode,
and fine-tune `aetheria_soul.pt` on a GPU so the training is fast and deep.

**Workflow:**
1. Collect feedback locally: `python Original_sin/envy/envy.py --mode teach`
2. Push to GitHub: `git add data/feedback.jsonl models/aetheria_soul.pt && git commit && git push`
3. Run this notebook on Colab (T4 GPU)
4. Download the new `.pt` → replace your local `models/aetheria_soul.pt`

**Before running:** `Runtime → Change runtime type → T4 GPU`

---

## Step 1 — Clone repo from GitHub
Change `GITHUB_REPO` to your repo URL.

In [ ]:
# ── Edit this to your GitHub repo ─────────────────────────────────────────
GITHUB_REPO = 'https://github.com/YOUR_USERNAME/AetheriaAI.git'
# ──────────────────────────────────────────────────────────────────────────

import os

if not os.path.exists('/content/AetheriaAI'):
    !git clone {GITHUB_REPO} /content/AetheriaAI
else:
    !cd /content/AetheriaAI && git pull

AETHERIA_ROOT = '/content/AetheriaAI/Aetheria'
os.chdir(AETHERIA_ROOT)
print('Working directory:', os.getcwd())
print('Files:', os.listdir('.'))

### Alternative — Mount Google Drive instead
If you keep your project on Drive, run this cell instead of the GitHub one above.
Skip if you used GitHub.

In [ ]:
# SKIP if you used GitHub above
# from google.colab import drive
# drive.mount('/content/drive')
# import os
# AETHERIA_ROOT = '/content/drive/MyDrive/AetheriaAI/Aetheria'
# os.chdir(AETHERIA_ROOT)
# print('Working directory:', os.getcwd())

## Step 2 — Install dependencies

In [ ]:
!pip install -q sentencepiece transformers accelerate
print('Done.')

## Step 3 — Check GPU

In [ ]:
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Device: {device}')
if device == 'cuda':
    print(torch.cuda.get_device_name(0))
    print(f'VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB')
else:
    print('WARNING: No GPU. Go to Runtime → Change runtime type → T4 GPU')

## Step 4 — Check feedback.jsonl and checkpoint

If using GitHub, this should already be there after `git pull`.
If your feedback isn't in the repo, use the upload cell below.

In [ ]:
import os, json

feedback_path = 'data/feedback.jsonl'
ckpt_path     = 'models/aetheria_soul.pt'
spm_path      = 'data/spm.model'

# Count feedback pairs
if os.path.exists(feedback_path):
    pairs = [json.loads(l) for l in open(feedback_path, encoding='utf-8') if l.strip()]
    print(f'feedback.jsonl : {len(pairs)} pairs')
    print(f'  Sample: Q={pairs[0]["human"][:50]}  A={pairs[0]["aetheria"][:50]}')
else:
    print('feedback.jsonl NOT FOUND — upload it below')

# Check checkpoint
if os.path.exists(ckpt_path):
    size_mb = os.path.getsize(ckpt_path) / (1024*1024)
    print(f'Checkpoint     : {ckpt_path}  ({size_mb:.1f} MB)')
else:
    print('aetheria_soul.pt NOT FOUND — upload it below')

# Check SPM tokenizer
print(f'spm.model      : {"OK" if os.path.exists(spm_path) else "NOT FOUND"}')

### Manual upload (only if files are missing after git pull)
Run this to upload `feedback.jsonl`, `aetheria_soul.pt`, and `spm.model` directly.

In [ ]:
# SKIP if all files were found above
# from google.colab import files as colab_files
# import shutil, os
#
# print('Upload: feedback.jsonl, aetheria_soul.pt, spm.model, spm.vocab')
# uploaded = colab_files.upload()
# for fname, data in uploaded.items():
#     if 'feedback' in fname:
#         dest = 'data/feedback.jsonl'
#     elif fname.endswith('.pt'):
#         dest = 'models/' + fname
#     elif 'spm.model' in fname:
#         dest = 'data/spm.model'
#     elif 'spm.vocab' in fname:
#         dest = 'data/spm.vocab'
#     else:
#         dest = fname
#     os.makedirs(os.path.dirname(dest) or '.', exist_ok=True)
#     with open(dest, 'wb') as f:
#         f.write(data)
#     print(f'Saved: {dest}')

## Step 5 — Retrain on feedback

Fine-tunes `aetheria_soul.pt` on your approved pairs.

**Tuning guide:**
- `EPOCHS`: 10–20 for <100 pairs, 5–10 for 200+ pairs
- `LR`: `3e-5` is safe; go up to `1e-4` if loss barely moves
- `BATCH_SIZE`: 16–32 on T4

In [ ]:
# ── Tune these ────────────────────────────────────────────────────────────
EPOCHS      = 15
LR          = 3e-5
BATCH_SIZE  = 16
SEQ_LEN     = 64
CKPT_IN     = 'models/aetheria_soul.pt'
CKPT_OUT    = 'models/aetheria_soul.pt'   # overwrite base (safe — git has the original)
# ─────────────────────────────────────────────────────────────────────────

!python scripts/retrain_feedback.py \
    --ckpt   {CKPT_IN} \
    --out    {CKPT_OUT} \
    --epochs {EPOCHS} \
    --lr     {LR} \
    --batch_size {BATCH_SIZE} \
    --seq_len    {SEQ_LEN}

## Step 6 — (Optional) Blind Devour on top

Run this **after** feedback retrain to also absorb DialoGPT-small's conversational
flow. Skip if you just want the feedback-only model.

Takes ~15 min on T4.

In [ ]:
# OPTIONAL — skip if you only want feedback retrain
# !python Original_sin/aetheria_core.py gluttony \
#     --gluttony_mode blind_devour \
#     --gluttony_model dialogpt-small \
#     --ckpt models/aetheria_soul.pt \
#     --epochs 3 \
#     --conversations 30 \
#     --turns 6 \
#     --pride_weight 0.4

## Step 7 — Quick sanity test

Run a few prompts through the model to see if it's coherent before downloading.

In [ ]:
import sys, torch, math
import torch.nn as nn
import sentencepiece as spm

sys.path.insert(0, 'scripts')

# ── Minimal inline model (mirrors lust.py) ────────────────────────────────
class _PE(nn.Module):
    def __init__(self, d, mx=1024):
        super().__init__()
        pe = torch.zeros(mx, d)
        pos = torch.arange(0, mx, dtype=torch.float).unsqueeze(1)
        div = torch.exp(torch.arange(0, d, 2).float() * (-math.log(10000.0) / d))
        pe[:, 0::2] = torch.sin(pos * div)
        pe[:, 1::2] = torch.cos(pos * div)
        self.register_buffer('pe', pe.unsqueeze(0))
    def forward(self, x): return x + self.pe[:, :x.size(1), :]

class TinyLM(nn.Module):
    def __init__(self, v, d=256, h=4, L=4, ff=1024):
        super().__init__()
        self.emb = nn.Embedding(v, d)
        self.pe  = _PE(d)
        enc = nn.TransformerEncoderLayer(d, h, ff, batch_first=True)
        self.tr = nn.TransformerEncoder(enc, L)
        self.ln = nn.LayerNorm(d)
        self.head = nn.Linear(d, v)
    def forward(self, x):
        e = self.pe(self.emb(x))
        return self.head(self.ln(self.tr(e.transpose(0,1)).transpose(0,1)))

def generate(model, tok, prompt, max_new=80, top_p=0.85, temp=0.75, rep=1.5):
    v = len(tok)
    ids = tok.encode(prompt)[-512:]
    model.eval()
    with torch.no_grad():
        for _ in range(max_new):
            x = torch.tensor([ids])
            logits = model(x)[0, -1]
            if logits.size(0) > v: logits[v:] = float('-inf')
            # rep penalty
            for g in set(ids[-64:]):
                if 0 <= g < logits.size(0):
                    logits[g] = logits[g]/rep if logits[g]>0 else logits[g]*rep
            logits = logits / temp
            probs = torch.softmax(logits, -1)
            sp, si = torch.sort(probs, descending=True)
            cum = torch.cumsum(sp, 0)
            keep = (cum - sp) < top_p; keep[0] = True
            mask = torch.zeros_like(probs, dtype=torch.bool)
            mask[si[keep]] = True
            probs = probs * mask; probs = probs / probs.sum()
            nxt = int(torch.multinomial(probs, 1).item())
            ids.append(max(0, min(nxt, v-1)))
    return tok.decode([max(0, min(i, v-1)) for i in ids])

def reply(model, tok, question):
    prompt = f'Human: {question}\nAetheria:'
    full   = generate(model, tok, prompt)
    text   = full[len(prompt):].strip()
    if 'Human:' in text: text = text.split('Human:')[0].strip()
    return text

# ── Load ──────────────────────────────────────────────────────────────────
ckpt = torch.load('models/aetheria_soul.pt', map_location='cpu', weights_only=False)
vs   = ckpt.get('vocab_size', 0)
st   = ckpt.get('model_state_dict', ckpt)
if vs <= 0:
    for k in ('token_emb.weight', 'embedding.weight'):
        if k in st: vs = st[k].shape[0]; break

tok = spm.SentencePieceProcessor()
tok.Load('data/spm.model')

m = TinyLM(vs)
ns = m.state_dict()
for k, v in ns.items():
    if k in st:
        vc = st[k]
        if vc.shape == v.shape: ns[k] = vc
        elif k.endswith(('emb.weight','head.weight','head.bias')):
            n = min(vc.shape[0], v.shape[0]); ns[k][:n] = vc[:n]
m.load_state_dict(ns)

# ── V1 readiness tests ────────────────────────────────────────────────────
TEST_PROMPTS = [
    'hello',
    'who are you?',
    'are you god?',
    'how do you do?',
    'what are you made of?',
    'do you feel emotions?',
]

print('='*60)
print('  V1 READINESS TEST')
print('='*60)
for q in TEST_PROMPTS:
    r = reply(m, tok, q)
    print(f'  Q: {q}')
    print(f'  A: {r}')
    print()

## Step 8 — Push updated model back to GitHub

In [ ]:
# Push the retrained checkpoint back to your repo
# Fill in your GitHub email + name if git complains

!git config user.email 'you@example.com'
!git config user.name  'Aetheria Colab'

!git add models/aetheria_soul.pt
!git commit -m 'Colab retrain: feedback baked in'

# For private repos you need a personal access token:
# !git remote set-url origin https://YOUR_TOKEN@github.com/YOUR_USERNAME/AetheriaAI.git

!git push
print('Pushed. Pull locally with: git pull')

### Alternative — Download directly to your browser
If you don't use GitHub, run this to download the checkpoint.

In [ ]:
# SKIP if you used git push above
# from google.colab import files as colab_files
# colab_files.download('models/aetheria_soul.pt')
# print('Downloaded. Place at: Aetheria/models/aetheria_soul.pt')

---
## Done!

**Local next steps:**
```powershell
git pull                         # get the retrained model
python Original_sin/aetheria_core.py talk --ckpt models/aetheria_soul.pt
```

**Loop:**
```
Local: python Original_sin/envy/envy.py --mode teach   (add 20-40 pairs)
       git add data/feedback.jsonl && git commit && git push
Colab: git pull → run Step 5 → git push
Local: git pull → talk
```